In [1]:
import math
import numpy as np

In [2]:
# Given values
sigma = 0.28
S0 = 37
K = 40
T = 0.75
r = 0.04
q = 0.02
alpha_temp = 5
M = 256

P_amer_bin = 5.0455539623

# Computing tau_final and the bounds for x
tau_final = (T * (sigma**2)) / 2
x_left = (math.log(S0/K)) + ((r - q - ((sigma**2)/2))*T) - (3*sigma*math.sqrt(T))
x_right = (math.log(S0/K)) + ((r - q - ((sigma**2)/2))*T) + (3*sigma*math.sqrt(T))

# Finite difference discretization
delta_tau =( tau_final / M)
N = math.floor((x_right - x_left) / math.sqrt(delta_tau / alpha_temp))
delta_x = (x_right - x_left) / N
alpha = delta_tau / (delta_x**2)

print(f"tau_final: {tau_final}, x_left: {x_left}, x_right: {x_right}")
print(f"delta_tau: {delta_tau}, N: {N}, delta_x: {delta_x}, alpha: {alpha}")

# Variables for transformation
a = ((r - q) / (sigma**2)) - 0.5
b = (((r - q) / (sigma**2)) + 0.5)**2 + ((2*q)/(sigma**2))

x_compute = math.log(S0/K)

tau_final: 0.029400000000000003, x_left: -0.8198228806486403, x_right: 0.6350997977092168
delta_tau: 0.00011484375000000001, N: 303, delta_x: 0.004801725011082036, alpha: 4.980957031250001


In [3]:
def f(x):
    return K * math.exp(a*x) * max(1 - math.exp(x), 0)

def g_left(tau):
    return K * math.exp((a*x_left) + (b*tau)) * (1 - math.exp(x_left))

def g_right(tau):
    return 0


In [4]:
def forward_subst(L,b):
    
    N = len(L)
    
    x = np.zeros(N)
    
    x[0] = b[0]/L[0][0]
    
    for j in range(1,N):
        row_sum = 0
        for k in range(j):
            row_sum += L[j][k]*x[k]
        x[j] = (b[j] - row_sum)/L[j][j]
    
    return x

In [5]:
def SOR(A , b , x0 , t1 , t2 , tol1 , tol2 , w  ):
    
    N = len(A)
    
    L_A = np.tril(A)
    U_A = np.triu(A)
    
    D_A = np.zeros((N,N))
    
    for i in range(N):
        D_A[i][i] = A[i][i]
        L_A[i][i] = 0
        U_A[i][i] = 0
    
    c = w*forward_subst(D_A + (w*L_A) , b)
    
    x1 = np.full(N,math.inf)
    
    xprev = x0
    x_new = np.full(N,math.inf)
    _iter = 0
    #residual = np.linalg.norm(b - np.dot(A,x0))
    #consec = np.linalg.norm(x1 - x0)
    if t1 and t2:
        while np.linalg.norm(x_new - x0) > tol1 or np.linalg.norm(b - np.dot(A,xprev)) > tol2:
            _iter += 1
            x0 = xprev
            x_new = forward_subst(D_A + (w*L_A) , np.dot((1-w)*D_A - (w*U_A), x0)) + c
            xprev = x_new
    elif t1:
        while np.linalg.norm(x_new - x0) > tol1: #or np.linalg.norm(b - np.dot(A,xprev)) > tol2:
            _iter += 1
            x0 = xprev
            x_new = forward_subst(D_A + (w*L_A) , np.dot((1-w)*D_A - (w*U_A), x0)) + c
            xprev = x_new
    elif t2:
        while np.linalg.norm(b - np.dot(A,xprev)) > tol2:
            _iter += 1
            x0 = xprev
            x_new = forward_subst(D_A + (w*L_A) , np.dot((1-w)*D_A - (w*U_A), x0)) + c
            xprev = x_new
    print(_iter) 
    return x_new

In [6]:
ee = [[0 for j in range(N+1)] for i in range(M+1)]

for m in range(1 , M+1):
    for i in range(1, N-1 +1):
        t = x_left + (i*delta_x)
        ttau = m*delta_tau
        
        ee[m][i] = K * math.exp(a*t + b*ttau) * max(1 - math.exp(t) , 0)

In [7]:
import pandas as pd
pd.DataFrame(ee)

,0,1,2,3,4,5,6,7,8,9,...,294,295,296,297,298,299,300,301,302,303
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0,27.223338,27.087408,26.951259,26.814892,26.678305,26.541496,26.404466,26.267212,26.129733,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0,27.226716,27.090769,26.954604,26.818219,26.681615,26.544790,26.407742,26.270471,26.132975,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0,27.230094,27.094130,26.957948,26.821547,26.684926,26.548084,26.411019,26.273731,26.136218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0,27.233473,27.097492,26.961293,26.824875,26.688237,26.551378,26.414296,26.276991,26.139461,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,0,28.084491,27.944261,27.803806,27.663125,27.522217,27.381081,27.239716,27.098120,26.956292,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
253,0,28.087976,27.947728,27.807256,27.666557,27.525632,27.384479,27.243096,27.101482,26.959637,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
254,0,28.091461,27.951196,27.810706,27.669990,27.529048,27.387877,27.246476,27.104845,26.962983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
255,0,28.094947,27.954664,27.814157,27.673424,27.532463,27.391275,27.249857,27.108208,26.966328,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [8]:
def crank_nicolson(M):
    
    delta_tau = tau_final / M
    N = math.floor((x_right - x_left) / math.sqrt(delta_tau / alpha_temp))
    delta_x = (x_right - x_left) / N
    alpha = delta_tau / (delta_x ** 2)
    
    print('M =', M)
    print('N =', N)
    print('Delta_tau =', delta_tau)
    print('Delta_x =', delta_x)
    print('Alpha =', alpha)
    
    U = [[0 for j in range(N+1)] for i in range(M+1)]

    # Initial condition
    for j in range(N+1):
        U[0][j] = f(x_left + j*delta_x)
    
    
    # Left Boundary condtion
    for i in range(1 , M+1):
        U[i][0] = g_left(i*delta_tau)
    
    #Right Boundary Condtion
    for i in range(1 , M+1):
        U[i][N] = g_right(i*delta_tau)
        
    # Boundary conditions are handled during the implicit step
    # Create the tridiagonal matrix and the solution vector
    A = np.zeros((N-1, N-1))
    B = np.zeros(N-1)

    # Main loop for the Crank-Nicolson scheme
    for i in range(0, M):

        # Set up the tridiagonal system
        for j in range(1, N):
            if j > 1:
                A[j-1, j-2] = -0.5 * alpha
            A[j-1, j-1] = 1 + alpha
            if j < N - 1:
                A[j-1, j] = -0.5 * alpha
            
            B[j-1] = (0.5 * alpha * U[i][j-1]) + ((1 - alpha) * U[i][j]) + (0.5 * alpha * U[i][j+1])
        
        B[0] += (alpha/2)*U[i+1][0]
        B[-1] += (alpha/2)*U[i+1][N]
        
        # Solve the tridiagonal system
        U_next = SOR(A,B, U[i][1:N],True , False, 1e-6 , 1e-6 , 1.2)
        
        # Addition for early exercise 
        
        # Update the solution
        for j in range(1, N):
            U[i+1][j] = max(U_next[j-1],ee[i+1][j])

        # Boundary conditions
        #U[i+1][0] = g_left((i+1) * delta_tau)
        #U[i+1][N] = g_right((i+1) * delta_tau)

    return U

U = crank_nicolson(M)


M = 256
N = 303
Delta_tau = 0.00011484375000000001
Delta_x = 0.004801725011082036
Alpha = 4.980957031250001
24
24
24
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21


In [9]:
def g_left_ep(tau):
    return K * math.exp(a*x_left + b*tau) * (math.exp(-2*r*tau/sigma**2) - math.exp(x_left - 2*q*tau/sigma**2))

In [10]:
def crank_nicolson_ep(M):
    
    delta_tau = tau_final / M
    N = math.floor((x_right - x_left) / math.sqrt(delta_tau / alpha_temp))
    delta_x = (x_right - x_left) / N
    alpha = delta_tau / (delta_x ** 2)
    
    print('M =', M)
    print('N =', N)
    print('Delta_tau =', delta_tau)
    print('Delta_x =', delta_x)
    print('Alpha =', alpha)
    
    U = [[0 for j in range(N+1)] for i in range(M+1)]

    # Initial condition
    for j in range(N+1):
        U[0][j] = f(x_left + j*delta_x)
    
    
    # Left Boundary condtion
    for i in range(1 , M+1):
        U[i][0] = g_left_ep(i*delta_tau)
    
    #Right Boundary Condtion
    for i in range(1 , M+1):
        U[i][N] = g_right(i*delta_tau)
        
    # Boundary conditions are handled during the implicit step
    # Create the tridiagonal matrix and the solution vector
    A = np.zeros((N-1, N-1))
    B = np.zeros(N-1)

    # Main loop for the Crank-Nicolson scheme
    for i in range(0, M):

        # Set up the tridiagonal system
        for j in range(1, N):
            if j > 1:
                A[j-1, j-2] = -0.5 * alpha
            A[j-1, j-1] = 1 + alpha
            if j < N - 1:
                A[j-1, j] = -0.5 * alpha
            
            B[j-1] = (0.5 * alpha * U[i][j-1]) + ((1 - alpha) * U[i][j]) + (0.5 * alpha * U[i][j+1])
        
        B[0] += (alpha/2)*U[i+1][0]
        B[-1] += (alpha/2)*U[i+1][N]
        
        # Solve the tridiagonal system
        U_next = SOR(A,B, U[i][1:N],True , False, 1e-6 , 1e-6 , 1.2)
        
        # Addition for early exercise 
        
        # Update the solution
        for j in range(1, N):
            U[i+1][j] = U_next[j-1]

        # Boundary conditions
        #U[i+1][0] = g_left((i+1) * delta_tau)
        #U[i+1][N] = g_right((i+1) * delta_tau)

    return U

U_ep = crank_nicolson(M)

M = 256
N = 303
Delta_tau = 0.00011484375000000001
Delta_x = 0.004801725011082036
Alpha = 4.980957031250001
24
24
24
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21


In [11]:
import pandas as pd
pd.DataFrame(U)

,0,1,2,3,4,5,6,7,8,9,...,294,295,296,297,298,299,300,301,302,303
0,27.355658,27.219961,27.084047,26.947916,26.811565,26.674995,26.538204,26.401190,26.263953,26.126491,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
1,27.359052,27.223338,27.087408,26.951259,26.814892,26.678305,26.541496,26.404466,26.267212,26.129733,...,1.201800e-34,6.431750e-35,3.441716e-35,1.841179e-35,9.840532e-36,5.242550e-36,2.760583e-36,1.391401e-36,5.808280e-37,0.0
2,27.362447,27.226716,27.090769,26.954604,26.818219,26.681615,26.544790,26.407742,26.270471,26.132975,...,9.305568e-33,5.025697e-33,2.713800e-33,1.464860e-33,7.898271e-34,4.243025e-34,2.250723e-34,1.140498e-34,4.770201e-35,0.0
3,27.365842,27.230094,27.094130,26.957948,26.821547,26.684926,26.548084,26.411019,26.273731,26.136218,...,3.493545e-31,1.902592e-31,1.035916e-31,5.637709e-32,3.064344e-32,1.659054e-32,8.864109e-33,4.518878e-33,1.897602e-33,0.0
4,27.369238,27.233473,27.097492,26.961293,26.824875,26.688237,26.551378,26.414296,26.276991,26.139461,...,8.605211e-30,4.725151e-30,2.593808e-30,1.423050e-30,7.796487e-31,4.253568e-31,2.288908e-31,1.173985e-31,4.950609e-32,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,28.224498,28.084491,27.944261,27.803806,27.663125,27.522217,27.381081,27.239716,27.098120,26.956292,...,1.449618e-02,1.277624e-02,1.109507e-02,9.447684e-03,7.829171e-03,6.234648e-03,4.659271e-03,3.098229e-03,1.546730e-03,0.0
253,28.228000,28.087976,27.947728,27.807256,27.666557,27.525632,27.384479,27.243096,27.101482,26.959637,...,1.471517e-02,1.297007e-02,1.126402e-02,9.592026e-03,7.949119e-03,6.330385e-03,4.730945e-03,3.145950e-03,1.570572e-03,0.0
254,28.231503,28.091461,27.951196,27.810706,27.669990,27.529048,27.387877,27.246476,27.104845,26.962983,...,1.493580e-02,1.316535e-02,1.143426e-02,9.737465e-03,8.069981e-03,6.426854e-03,4.803168e-03,3.194038e-03,1.594598e-03,0.0
255,28.235006,28.094947,27.954664,27.814157,27.673424,27.532463,27.391275,27.249857,27.108208,26.966328,...,1.515806e-02,1.336209e-02,1.160576e-02,9.883996e-03,8.191755e-03,6.524054e-03,4.875940e-03,3.242491e-03,1.618806e-03,0.0


In [12]:
from scipy.stats import norm

def black_scholes_put(S0, K, r, q, T, sigma):
    d1 = (math.log(S0/K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    put_price = K * math.exp(-r * T) * norm.cdf(-d2) - S0 * math.exp(-q * T) * norm.cdf(-d1)
    return put_price
    
#N = 22
def pointwise_convergence(U):
    x_compute = math.log(S0/K)
    print('x_compute = ' , x_compute)
    for i in range(N):
        if (x_left + (i*delta_x) <= x_compute)  and  (x_compute < x_left + (i+1)*delta_x):
            print(i)
            
            xi, xi1 = x_left + (i*delta_x), x_left + ((i+1)*delta_x)
            Si, Si1 = K*math.exp(xi), K*math.exp(xi1)
            print('S_left = ' , Si)
            print('S_right = ' , Si1)
            Vi, Vi1 = math.exp(-a*xi - b*tau_final)*U[M][i], math.exp(-a*xi1 - b*tau_final)*U[M][i+1]
            V_approx = (((Si1 - S0)*Vi) + ((S0 - Si)*Vi1)) / (Si1 - Si)
            V_exact = P_amer_bin
            error_pointwise = abs(V_approx - V_exact) #/ V_exact

            # Interpolation
            u_xcompute = (((xi1 - x_compute)*U[M][i]) + ((x_compute - xi)*U[M][i+1]) )/ (xi1 - xi)
            V_approx2 = math.exp(-a*x_compute - b*tau_final)*u_xcompute
            error_pointwise2 = abs(V_approx2 - V_exact) #/ V_exact

            return error_pointwise, error_pointwise2, V_approx

error_pointwise, error_pointwise2, Vapprox = pointwise_convergence(U)
error_pointwise_ep, error_pointwise2_ep, Vapprox_ep = pointwise_convergence(U_ep)
print(Vapprox)

x_compute =  -0.0779615414697118
154
S_left =  36.911465656178166
S_right =  37.0891305716876
x_compute =  -0.0779615414697118
154
S_left =  36.911465656178166
S_right =  37.0891305716876
5.045211248593274


In [13]:
BS = black_scholes_put(S0, K, r, q, T, sigma)

In [14]:
error_pointwise, error_pointwise2

(0.00034271370672556145, 0.0003726129287233704)

In [15]:
#X-grid
x = []
S = []
for j in range(N+1):
    t = x_left + j*delta_x
    x.append(t)
    S.append(K*math.exp(t))

In [16]:
S

[17.620386815352823,
 17.70519852572903,
 17.790418457916278,
 17.87604857679708,
 17.962090856711466,
 18.048547281502504,
 18.13541984456204,
 18.222710548876645,
 18.31042140707382,
 18.39855444146839,
 18.487111684109124,
 18.57609517682559,
 18.665506971275253,
 18.755349128990744,
 18.845623721427426,
 18.936332830011132,
 19.027478546186167,
 19.119062971463514,
 19.211088217469317,
 19.303556405993547,
 19.396469669038908,
 19.489830148870038,
 19.583639998062864,
 19.677901379554243,
 19.772616466691844,
 19.867787443284243,
 19.963416503651278,
 20.059505852674654,
 20.156057705848763,
 20.25307428933177,
 20.35055783999696,
 20.44851060548428,
 20.54693484425219,
 20.645832825629714,
 20.74520682986879,
 20.845059148196803,
 20.945392082869464,
 21.046207947223856,
 21.147509065731775,
 21.24929777405334,
 21.351576419090836,
 21.45434735904282,
 21.557612963458514,
 21.661375613292414,
 21.765637700959193,
 21.87040163038887,
 21.975669817082238,
 22.08144468816655,
 22.187

In [17]:
def compute_rms_error(U, S, a, b, tau_final, K, r, q, sigma, S0, threshold=0.00001):
    
    V_approx = [math.exp(-(a * x[i]) - (b * tau_final)) * U[-1][i] for i in range(len(x))]
    V_exact = [black_scholes_put(S[i], K, r, q,T , sigma) for i in range(len(S))]
    
    valid_indices = [i for i, val in enumerate(V_exact) if val > threshold * S0]
    NRMS = len(valid_indices)
    
    print(valid_indices )
    print(V_exact)
    errors = [((V_approx[i] - V_exact[i])**2) /(V_exact[i]**2) for i in valid_indices]
    rms_error = math.sqrt(sum(errors) / NRMS)
    return rms_error

In [18]:
def compute_greeks(U, S, a, b, tau_final, delta_tau, delta_x, i):
    # Values of option at nodes of interest
    Vi = math.exp(-a * x[i] - b * tau_final) * U[-1][i]
    Vi_plus_1 = math.exp(-a * x[i+1] - b * tau_final) * U[-1][i+1]
    Vi_minus_1 = math.exp(-a * x[i-1] - b * tau_final) * U[-1][i-1]
    Vi_plus_2 = math.exp(-a * x[i+2] - b * tau_final) * U[-1][i+2]
    Vi_delta_t = math.exp(-a * x[i] - b * (tau_final - delta_tau)) * U[-2][i]
    Vi1_delta_t = math.exp(-a * x[i+1] - b * (tau_final - delta_tau)) * U[-2][i+1]
    
    # Delta
    delta_fd = (Vi_plus_1 - Vi) / (S[i+1] - S[i])
    
    # Gamma
    gamma_fd = ((Vi_plus_2 - Vi_plus_1) / (S[i+2] - S[i+1])) - ((Vi - Vi_minus_1) / (S[i] - S[i-1])) 
    gamma_fd /= (S[i+2] + S[i+1])/2 - (S[i] + S[i-1])/2
    
    # Theta
    V_approx_current = (((S[i+1] - S0) * Vi) + ((S0 - S[i]) * Vi_plus_1)) / (S[i+1] - S[i])
    
    V_approx_previous = (((S[i+1] - S0) * Vi_delta_t) + ((S0 - S[i]) * Vi1_delta_t)) / (S[i+1] - S[i])
    
    delta_t = (2 * delta_tau) / (sigma**2)
    theta_fd = (V_approx_previous - V_approx_current) / delta_t

    return delta_fd, gamma_fd, theta_fd


In [19]:
rms_error = compute_rms_error(U, S, a, b, tau_final, K, r, q, sigma, S0)
print(f"RMS Error: {rms_error}")

# Find i such that xi <= xcompute < xi+1
i = (next(idx for idx, val in enumerate(x) if val > x_compute)) - 1
delta_fd, gamma_fd, theta_fd = compute_greeks(U, S, a, b, tau_final, delta_tau, delta_x, i)

print('I = ' , i )

print(f"Delta (Δ): {delta_fd}")
print(f"Gamma (Γ): {gamma_fd}")
print(f"Theta (Θ): {theta_fd}")


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [20]:
print("{:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f}".format(error_pointwise,  error_pointwise2,   rms_error, delta_fd, gamma_fd, theta_fd))


0.000343 0.000373 0.124610 -0.565583 0.046536 -1.879042


In [21]:
# Round values to maintain 6 digits after the decimal
error_pointwise = round(error_pointwise, 6)
error_pointwise2 = round(error_pointwise2, 6)
rms_error = round(rms_error, 6)
delta_fd = round(delta_fd, 6)
gamma_fd = round(gamma_fd, 6)
theta_fd = round(theta_fd, 6)

# Create a DataFrame
data = {
    'error_pointwise': [error_pointwise],
    'gap': [''],  # gap
    'error_pointwise2': [error_pointwise2],
    'gap2': [''],  # gap
    'delta_fd': [delta_fd],
    'gamma_fd': [gamma_fd],
    'theta_fd': [theta_fd],
    'var_red': [Vapprox + (BS - Vapprox_ep)],
    'var_red_pointwise': [abs(Vapprox + (BS - Vapprox_ep) - P_amer_bin)]
}
df = pd.DataFrame(data)

# Display the DataFrame
display(df)

,error_pointwise,gap,error_pointwise2,gap2,delta_fd,gamma_fd,theta_fd,var_red,var_red_pointwise
0,0.000343,,0.000373,,-0.565583,0.046536,-1.879042,4.937133,0.108421
